In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Consumo de Cimento (t)
0,2003-1,0.797768,0.311113,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.256606,20.216800,1.320065e+08,6.638363e+06,19.725168,1.221860e+08,142.456
1,2003-2,0.797853,0.312787,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.250731,19.631031,1.321743e+08,6.642080e+06,19.732277,1.222513e+08,135.829
2,2003-3,0.797939,0.314207,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.244856,19.400920,1.323421e+08,6.645797e+06,19.739386,1.223166e+08,133.135
3,2003-4,0.798024,0.315571,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.238982,19.153741,1.325098e+08,6.649513e+06,19.746495,1.223818e+08,144.723
4,2003-5,0.798110,0.316880,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.233107,18.912155,1.326776e+08,6.653230e+06,19.753604,1.224471e+08,150.523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.663849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.321494,NaN,NaN,NaN,NaN,377.545
236,2022-9,NaN,0.663153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.790929,NaN,NaN,NaN,NaN,370.282
237,2022-10,NaN,0.661797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.428377,NaN,NaN,NaN,NaN,340.900
238,2022-11,NaN,0.660090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.448086,NaN,NaN,NaN,NaN,363.900


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.726097,1.713104,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.126590,0.933479,1.196588,-0.578473,0.290543,1.029069
188,0.689854,1.722266,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.131368,0.953950,1.189460,-0.568205,0.281308,1.023160
189,0.653611,1.738852,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.136147,0.982622,1.182332,-0.557938,0.272072,1.017252
190,0.617368,1.755526,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.140925,1.015842,1.175204,-0.547670,0.262837,1.011344


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Santa Catarina - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.514485,1.376970,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.817925,0.890411,1.181211,-0.551568,0.108581,0.958937
158,1.496494,1.401965,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.835333,0.883759,1.187451,-0.577579,0.126542,0.968867
159,1.478503,1.424150,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.852741,0.877272,1.193692,-0.603589,0.144504,0.978796
160,1.460512,1.443460,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.870149,0.861580,1.199932,-0.629600,0.162465,0.988726


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
157    211.861
158    258.760
159    211.393
160    249.531
161    231.359
Name: Santa Catarina - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
126,0.730991,0.354460,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,-1.145081,1.261102,0.800980,0.503024,0.784774,0.894737
127,0.741997,0.396550,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,-1.124363,1.264193,0.819811,0.473034,0.769927,0.903886
128,0.753002,0.439226,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,-1.103645,1.264246,0.838643,0.443045,0.755079,0.913035
129,0.764008,0.479949,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,-1.082927,1.262191,0.857474,0.413055,0.740232,0.922184
130,0.775014,0.523578,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,-1.062209,1.252654,0.876306,0.383065,0.725384,0.931332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.726097,1.713104,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.126590,0.933479,1.196588,-0.578473,0.290543,1.029069
188,0.689854,1.722266,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.131368,0.953950,1.189460,-0.568205,0.281308,1.023160
189,0.653611,1.738852,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.136147,0.982622,1.182332,-0.557938,0.272072,1.017252
190,0.617368,1.755526,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.140925,1.015842,1.175204,-0.547670,0.262837,1.011344


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1157475764, 639114933, 1554052405, 1642055606, 211013631, 458283673, 3211199770, 3892594472, 2504149846, 2489019898, 252853362, 3867352579, 701312624, 3003314342, 2991835875, 293273594, 780628207, 843772721, 3454372106, 1455995079, 302404878, 1587632620, 2924277358, 2098721801, 3272675418, 310363774, 1752578818, 3140954649, 3816719732, 3973842685, 1357382712, 3861915095, 3224592031, 4188216746, 859443457, 2430371590, 4173024963, 2119012016, 4144332355, 3014003409, 1864085667, 1789515035, 1500746675, 3177109423, 2075224377, 3204337914, 413170455, 966402662, 1407791888, 3961607036]


Step: 0 ___________________________________________
val_loss: 229.46170043945312
winner_seed: 1157475764


Step: 1 ___________________________________________
val_loss: 315.42755126953125


Step: 2 ___________________________________________
val_loss: 328.517333984375


Step: 3 ___________________________________________
val_loss: 284.6405029296875


Step: 4 ___________________________________________
val_l

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 213ms/step - loss: 45052.5430 - val_loss: 37157.2969
Epoch 2/10000
4/4 [==============================] - 0s 13ms/step - loss: 114295.5625 - val_loss: 478372.1250
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 410131.9688 - val_loss: 5338.6621
Epoch 4/10000
4/4 [==============================] - 0s 13ms/step - loss: 4274.0137 - val_loss: 2230.5005
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 2306.2285 - val_loss: 1545.3032
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 1583.0868 - val_loss: 792.3771
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 863.3387 - val_loss: 794.1833
Epoch 8/10000
4/4 [==============================] - 0s 13ms/step - loss: 889.5289 - val_loss: 760.3461
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 695.1157 - val_loss: 738.0162
Epoch 10/10000
4/4 [========================

4/4 [==============================] - 0s 13ms/step - loss: 355.9382 - val_loss: 503.4941
Epoch 79/10000
4/4 [==============================] - 0s 13ms/step - loss: 374.5641 - val_loss: 516.9432
Epoch 80/10000
4/4 [==============================] - 0s 13ms/step - loss: 332.0358 - val_loss: 672.8977
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 332.6542 - val_loss: 435.3688
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 298.1197 - val_loss: 651.5686
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 429.8080 - val_loss: 534.4877
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 276.1202 - val_loss: 459.9383
Epoch 85/10000
4/4 [==============================] - 0s 12ms/step - loss: 267.9026 - val_loss: 495.4377
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 309.3453 - val_loss: 537.0607
Epoch 87/10000
4/4 [==============================] - 0s 13ms/step - l

4/4 [==============================] - 0s 13ms/step - loss: 385.9374 - val_loss: 541.3425
Epoch 156/10000
4/4 [==============================] - 0s 13ms/step - loss: 313.1328 - val_loss: 519.2800
Epoch 157/10000
4/4 [==============================] - 0s 12ms/step - loss: 288.1458 - val_loss: 478.4755
Epoch 158/10000
4/4 [==============================] - 0s 12ms/step - loss: 258.7145 - val_loss: 488.4830
Epoch 159/10000
4/4 [==============================] - 0s 13ms/step - loss: 290.4148 - val_loss: 500.5458
Epoch 160/10000
4/4 [==============================] - 0s 14ms/step - loss: 303.9437 - val_loss: 561.2753
Epoch 161/10000
4/4 [==============================] - 0s 13ms/step - loss: 333.7701 - val_loss: 616.6302
Epoch 162/10000
4/4 [==============================] - 0s 13ms/step - loss: 319.1664 - val_loss: 449.7037
Epoch 163/10000
4/4 [==============================] - 0s 13ms/step - loss: 322.6901 - val_loss: 830.1407
Epoch 164/10000
4/4 [==============================] - 0s 13ms

4/4 [==============================] - 0s 12ms/step - loss: 232.8554 - val_loss: 503.7602
Epoch 233/10000
4/4 [==============================] - 0s 12ms/step - loss: 276.4967 - val_loss: 528.8465
Epoch 234/10000
4/4 [==============================] - 0s 13ms/step - loss: 309.4143 - val_loss: 593.8811
Epoch 235/10000
4/4 [==============================] - 0s 13ms/step - loss: 256.9436 - val_loss: 514.1453
Epoch 236/10000
4/4 [==============================] - 0s 13ms/step - loss: 258.2198 - val_loss: 482.7681
Epoch 237/10000
4/4 [==============================] - 0s 13ms/step - loss: 247.7354 - val_loss: 483.7837
Epoch 238/10000
4/4 [==============================] - 0s 13ms/step - loss: 282.8494 - val_loss: 901.7630
Epoch 239/10000
4/4 [==============================] - 0s 14ms/step - loss: 353.9723 - val_loss: 566.7087
Epoch 240/10000
4/4 [==============================] - 0s 13ms/step - loss: 259.2476 - val_loss: 564.5191
Epoch 241/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 13ms/step - loss: 270.7264 - val_loss: 492.6315
Epoch 310/10000
4/4 [==============================] - 0s 13ms/step - loss: 371.0327 - val_loss: 482.9305
Epoch 311/10000
4/4 [==============================] - 0s 13ms/step - loss: 237.6599 - val_loss: 459.6703
Epoch 312/10000
4/4 [==============================] - 0s 13ms/step - loss: 240.0196 - val_loss: 462.8684
Epoch 313/10000
4/4 [==============================] - 0s 13ms/step - loss: 266.8179 - val_loss: 505.9468
Epoch 314/10000
4/4 [==============================] - 0s 14ms/step - loss: 308.1909 - val_loss: 555.2117
Epoch 315/10000
4/4 [==============================] - 0s 13ms/step - loss: 284.2215 - val_loss: 920.4326
Epoch 316/10000
4/4 [==============================] - 0s 13ms/step - loss: 332.8592 - val_loss: 492.9235
Epoch 317/10000
4/4 [==============================] - 0s 13ms/step - loss: 227.3358 - val_loss: 526.4876
Epoch 318/10000
4/4 [==============================] - 0s 13ms

...

4/4 [==============================] - 0s 13ms/step - loss: 97.1003 - val_loss: 307.0056
Epoch 2908/10000
4/4 [==============================] - 0s 13ms/step - loss: 114.4319 - val_loss: 282.5703
Epoch 2909/10000
4/4 [==============================] - 0s 14ms/step - loss: 101.5320 - val_loss: 293.0421
Epoch 2910/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.2228 - val_loss: 271.5576
Epoch 2911/10000
4/4 [==============================] - 0s 12ms/step - loss: 88.3353 - val_loss: 278.7443
Epoch 2912/10000
4/4 [==============================] - 0s 12ms/step - loss: 103.7749 - val_loss: 296.8421
Epoch 2913/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.0314 - val_loss: 347.2133
Epoch 2914/10000
4/4 [==============================] - 0s 12ms/step - loss: 95.3820 - val_loss: 294.4875
Epoch 2915/10000
4/4 [==============================] - 0s 11ms/step - loss: 77.3342 - val_loss: 300.4886
Epoch 2916/10000
4/4 [==============================] - 0s 1

4/4 [==============================] - 0s 12ms/step - loss: 83.7379 - val_loss: 194.1827
Epoch 2985/10000
4/4 [==============================] - 0s 12ms/step - loss: 107.0497 - val_loss: 263.4081
Epoch 2986/10000
4/4 [==============================] - 0s 12ms/step - loss: 121.2683 - val_loss: 239.9292
Epoch 2987/10000
4/4 [==============================] - 0s 12ms/step - loss: 112.1609 - val_loss: 239.5095
Epoch 2988/10000
4/4 [==============================] - 0s 13ms/step - loss: 94.9847 - val_loss: 416.6920
Epoch 2989/10000
4/4 [==============================] - 0s 12ms/step - loss: 110.8266 - val_loss: 227.0048
Epoch 2990/10000
4/4 [==============================] - 0s 13ms/step - loss: 96.4227 - val_loss: 235.4168
Epoch 2991/10000
4/4 [==============================] - 0s 12ms/step - loss: 94.7832 - val_loss: 247.6603
Epoch 2992/10000
4/4 [==============================] - 0s 12ms/step - loss: 85.4273 - val_loss: 251.4920
Epoch 2993/10000
4/4 [==============================] - 0s 

4/4 [==============================] - 0s 12ms/step - loss: 75.6649 - val_loss: 495.2926
Epoch 3062/10000
4/4 [==============================] - 0s 12ms/step - loss: 94.3499 - val_loss: 304.2173
Epoch 3063/10000
4/4 [==============================] - 0s 12ms/step - loss: 108.3399 - val_loss: 310.4234
Epoch 3064/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.7274 - val_loss: 360.9271
Epoch 3065/10000
4/4 [==============================] - 0s 12ms/step - loss: 84.6244 - val_loss: 337.1277
Epoch 3066/10000
4/4 [==============================] - 0s 13ms/step - loss: 83.9587 - val_loss: 293.4211
Epoch 3067/10000
4/4 [==============================] - 0s 12ms/step - loss: 85.8723 - val_loss: 251.1246
Epoch 3068/10000
4/4 [==============================] - 0s 13ms/step - loss: 83.9228 - val_loss: 250.8588
Epoch 3069/10000
4/4 [==============================] - 0s 13ms/step - loss: 92.7893 - val_loss: 401.6629
Epoch 3070/10000
4/4 [==============================] - 0s 13m

4/4 [==============================] - 0s 13ms/step - loss: 78.5708 - val_loss: 439.0498
Epoch 3139/10000
4/4 [==============================] - 0s 12ms/step - loss: 86.4836 - val_loss: 450.6747
Epoch 3140/10000
4/4 [==============================] - 0s 12ms/step - loss: 78.3680 - val_loss: 438.6267
Epoch 3141/10000
4/4 [==============================] - 0s 12ms/step - loss: 81.5132 - val_loss: 451.8207
Epoch 3142/10000
4/4 [==============================] - 0s 13ms/step - loss: 84.3102 - val_loss: 424.2829
Epoch 3143/10000
4/4 [==============================] - 0s 13ms/step - loss: 82.2279 - val_loss: 279.5497
Epoch 3144/10000
4/4 [==============================] - 0s 12ms/step - loss: 82.0392 - val_loss: 394.6178
Epoch 3145/10000
4/4 [==============================] - 0s 13ms/step - loss: 82.1004 - val_loss: 225.7569
Epoch 3146/10000
4/4 [==============================] - 0s 13ms/step - loss: 96.8051 - val_loss: 251.9333
Epoch 3147/10000
4/4 [==============================] - 0s 13ms

4/4 [==============================] - 0s 11ms/step - loss: 101.2883 - val_loss: 406.9202
Epoch 3216/10000
4/4 [==============================] - 0s 12ms/step - loss: 91.7512 - val_loss: 547.8220
Epoch 3217/10000
4/4 [==============================] - 0s 12ms/step - loss: 94.9455 - val_loss: 411.1261
Epoch 3218/10000
4/4 [==============================] - 0s 13ms/step - loss: 73.5415 - val_loss: 415.4806
Epoch 3219/10000
4/4 [==============================] - 0s 14ms/step - loss: 86.3439 - val_loss: 432.5440
Epoch 3220/10000
4/4 [==============================] - 0s 13ms/step - loss: 72.1757 - val_loss: 422.5308
Epoch 3221/10000
4/4 [==============================] - 0s 13ms/step - loss: 62.3805 - val_loss: 442.2403
Epoch 3222/10000
4/4 [==============================] - 0s 12ms/step - loss: 67.3459 - val_loss: 404.5154
Epoch 3223/10000
4/4 [==============================] - 0s 12ms/step - loss: 93.1491 - val_loss: 454.6007
Epoch 3224/10000
4/4 [==============================] - 0s 12m

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,235.091446,235.475494,236.267731,236.896133,237.04512,237.743896,238.678177,239.012329,239.986603,241.149963,241.818817,242.112671,242.775833,242.987381,243.235092,244.826767,245.081787,244.515778,243.531113,243.821701,244.839035,244.37117,244.155121,243.800171,243.474365,243.124374,242.393997,240.733749,239.817719,238.938156
Target,265.999,277.169,245.824,244.227,263.751,211.136,219.002,234.275,249.92,246.326,198.778,285.97,262.597,277.811,235.885,272.671,278.592,214.658,252.7,246.146,255.873,273.299,270.918,256.605,303.526,299.284,284.55,307.576,281.317,226.233
Error,30.907547,41.693512,9.556274,7.330872,26.705887,26.607895,19.676178,4.737335,9.933395,5.176041,43.040817,43.85733,19.821152,34.823624,7.350098,27.844223,33.510223,29.857773,9.168884,2.324295,11.033966,28.927841,26.762878,12.80484,60.051636,56.159622,42.155991,66.842239,41.499268,12.705154


In [38]:
display(mae)
display(mape)

26.428892

0.09983377

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[2861.2783]] - Target[2942.3769999999995]| =  Error: [[81.09863]]; MAPE:[[0.02756228]]
1/1 [==============================] - 0s 19ms/step
Ano-0: |Prediction[[2927.941]] - Target[3097.755]| =  Error: [[169.81396]]; MAPE:[[0.0548184]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[1448.4823]] - Target[1702.4859999999999]| =  Error: [[254.00366]]; MAPE:[[0.14919575]]


[array([[81.09863]], dtype=float32),
 array([[169.81396]], dtype=float32),
 array([[254.00366]], dtype=float32)]

168.30542

0.07719214